# Process Data

This default notebook is executed using Databricks Workflows as defined in resources/petrinex.job.yml.

In [1]:
from petrinex.config import load_config
from petrinex.process import process_csvs_to_delta
from databricks.connect import DatabricksSession as SparkSession
import os

In [18]:
# Initialize Spark session and load configuration
spark = SparkSession.builder.serverless(True).getOrCreate()
config = load_config('config.yaml')

In [ ]:
import logging
logger = logging.getLogger(__name__)

In [ ]:
# args
dataset = 'conventional'

csv_path_pattern = f"{config.ingest.download_dir}/{dataset}/*.CSV"
logger.info(f"Reading CSV files from pattern: {csv_path_pattern}")

INFO:__main__:Reading CSV files from pattern: Volumes/shm/petrinex/bronze/conventional/*.CSV


In [19]:
df = (
        spark.read.option("header", True)
        .option("inferSchema", False)
        .option("pathGlobFilter", "*.CSV")
        .csv(csv_path_pattern)
    )

In [20]:
df.limit(1).show()

{"ts": "2025-08-06 11:59:36.301", "level": "ERROR", "logger": "pyspark.sql.connect.client.logging", "msg": "GRPC Error received", "context": {}, "exception": {"class": "_MultiThreadedRendezvous", "msg": "<_MultiThreadedRendezvous of RPC that terminated with:\n\tstatus = StatusCode.INTERNAL\n\tdetails = \"Cannot access the UC Volume path from this location. Path was /Volumes/shm/petrinex/bronze/conventional\"\n\tdebug_error_string = \"UNKNOWN:Error received from peer  {grpc_status:13, grpc_message:\"Cannot access the UC Volume path from this location. Path was /Volumes/shm/petrinex/bronze/conventional\"}\"\n>", "stacktrace": [{"class": null, "method": "_execute_and_fetch_as_iterator", "file": "/Users/scott.mckean/Repos/Petrinex/.venv/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py", "line": "1915"}, {"class": null, "method": "__next__", "file": "<frozen _collections_abc>", "line": "356"}, {"class": null, "method": "send", "file": "/Users/scott.mckean/Repos/Petrinex/.venv

IllegalArgumentException: Cannot access the UC Volume path from this location. Path was /Volumes/shm/petrinex/bronze/conventional

JVM stacktrace:
java.lang.IllegalArgumentException
	at com.databricks.sql.acl.fs.volumes.UCVolumesFileSystem.initializeVolumePath(UCVolumesFileSystem.scala:98)
	at com.databricks.sql.acl.fs.volumes.UCVolumesFileSystem.getReadDelegate(UCVolumesFileSystem.scala:131)
	at com.databricks.sql.acl.fs.volumes.UCVolumesFileSystem.listStatus(UCVolumesFileSystem.scala:231)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$2(DatabricksFileSystemV2.scala:178)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:64)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$1(DatabricksFileSystemV2.scala:175)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:510)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogg

In [ ]:

    

    # Read CSV files into DataFrame
    

    # Create catalog and schema if they don't exist
    logger.info(
        f"Ensuring catalog {config.catalog} and schema {config.schema_name} exist"
    )
    spark.sql(f"CREATE CATALOG IF NOT EXISTS {config.catalog}")
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {config.catalog}.{config.schema_name}")

    # Get full table name
    full_table_name = f"{config.catalog}.{config.schema_name}.{dataset.table_name}"

    # Write DataFrame to Delta table
    logger.info(f"Writing DataFrame to Delta table: {full_table_name}")
    df.write.option("overwriteSchema", "true").mode("overwrite").format(
        "delta"
    ).saveAsTable(full_table_name)

    row_count = df.count()
    logger.info(
        f"{dataset.name} data successfully written to {full_table_name} ({row_count:,} rows)"
    )

    return df

IndentationError: unexpected indent (2516325597.py, line 8)

In [ ]:
# Process conventional data if CSV files exist
conventional_csv_dir = os.path.join(config.ingest.download_dir, config.ingest.conventional.download_subdir)
if os.path.exists(conventional_csv_dir) and any(f.endswith('.CSV') for f in os.listdir(conventional_csv_dir)):
    print(f"Processing conventional data from {conventional_csv_dir}")
    conventional_df = process_csvs_to_delta(config, config.ingest.conventional, conventional_csv_dir, spark)
    print(f"Conventional data processed successfully: {conventional_df.count()} rows")
else:
    print(f"No conventional CSV files found in {conventional_csv_dir}")


In [ ]:
# Process NGL data if CSV files exist
ngl_csv_dir = os.path.join(config.ingest.download_dir, config.ingest.ngl.download_subdir)
if os.path.exists(ngl_csv_dir) and any(f.endswith('.CSV') for f in os.listdir(ngl_csv_dir)):
    print(f"Processing NGL data from {ngl_csv_dir}")
    ngl_df = process_csvs_to_delta(config, config.ingest.ngl, ngl_csv_dir, spark)
    print(f"NGL data processed successfully: {ngl_df.count()} rows")
else:
    print(f"No NGL CSV files found in {ngl_csv_dir}")
